# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# Inicia a Variável Navegador que recebe o webdriver do chrome para abrí-lo
navegador = webdriver.Chrome("/home/anderosn/anaconda3/chromedriver")

# Usa o método get do webdriver para entrar na página passada no argumento da função
navegador.get("https://www.google.com.br/")

# Procura o elemento do XPATH passado como parâmetro e em seguida chama a função que envia o texto passado no parâmetro do send_keys
# Dólar
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input').send_keys("cotação do dolar")

navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input').send_keys(Keys.ENTER)

dolar = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(dolar)

# Euro
navegador.get("https://www.google.com.br/")

navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input').send_keys("cotação do euro")

navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input').send_keys(Keys.ENTER)

euro = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(euro)

# Ouro

navegador.get("https://www.melhorcambio.com/ouro-hoje")

ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')
ouro = ouro.replace(',','.')
print(ouro)

navegador.quit()

/tmp/ipykernel_34537/204063183.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  navegador = webdriver.Chrome("/home/anderosn/anaconda3/chromedriver")


5.520099999999999
6.3335
323.26


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [16]:
#importar base de dados
import pandas as pd

tabela = pd.read_excel('Produtos.xlsx')

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [19]:
# Atualiza a base de dados
tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(dolar)
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = float(euro)
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = float(ouro)

# Atualiza o cálculo de valores

tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']

tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.5201,5520.044799,1.40,7728.062719
1,Carro Renault,4500.00,Euro,6.3335,28500.750000,2.00,57001.500000
2,Notebook Dell,899.99,Dólar,5.5201,4968.034799,1.70,8445.659158
3,IPhone,799.00,Dólar,5.5201,4410.559900,1.70,7497.951830
4,Carro Fiat,3000.00,Euro,6.3335,19000.500000,1.90,36100.950000
5,Celular Xiaomi,480.48,Dólar,5.5201,2652.297648,2.00,5304.595296
6,Joia 20g,20.00,Ouro,323.2600,6465.200000,1.15,7434.980000


### Agora vamos exportar a nova base de preços atualizada

In [20]:
tabela.to_excel('Novo Produtos.xlsx', index=False)